# Advanced image processes Ⅱ

&copy; 李浩东 3190104890@zju.edu.cn
&copy; 孙正男 3190104890@zju.edu.cn

- Contour finding
- Contour approximation

# Contour extraction

![000](./images/contour_extraction.png)
